# Q3: Even deeper! Resnet18 for PASCAL classification (15 pts)

Hopefully we all got much better accuracy with the deeper model! Since 2012, much deeper architectures have been proposed. [ResNet](https://arxiv.org/abs/1512.03385) is one of the popular ones. In this task, we attempt to further improve the performance with the “very deep” ResNet-18 architecture.


## 3.1 Build ResNet-18 (1 pts)
Write a network modules for the Resnet-18 architecture (refer to the original paper). You can use `torchvision.models` for this section, so it should be very easy!

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import matplotlib.pyplot as plt
%matplotlib inline

import trainer
from utils import ARGS
from simple_cnn import SimpleCNN
from voc_dataset import VOCDataset


# you could write the whole class....
# or one line :D
ResNet = models.resnet18()
ResNet.fc = nn.Linear(512,20,bias=True)
print(ResNet.fc.bias)

Parameter containing:
tensor([-0.0318, -0.0123,  0.0417, -0.0207,  0.0186, -0.0298, -0.0212,  0.0408,
        -0.0237,  0.0074,  0.0237,  0.0320,  0.0023, -0.0410,  0.0283,  0.0095,
        -0.0364,  0.0172, -0.0391, -0.0114], requires_grad=True)


## 3.2 Add Tensorboard Summaries (6 pts)
You should've already written tensorboard summary generation code into `trainer.py` from q1. However, you probably just added the most basic summary features. Please implement the more advanced summaries listed here:
* training loss (should be done)
* testing MAP curves (should be done)
* learning rate
* histogram of gradients

## 3.3 Train and Test (8 pts)
Use the same hyperparameter settings from Task 2, and train the model for 50 epochs. Report tensorboard screenshots for *all* of the summaries listed above (for image summaries show screenshots at $n \geq 3$ iterations)

**REMEMBER TO SAVE A MODEL AT THE END OF TRAINING**

In [3]:
args = ARGS(batch_size=32, test_batch_size=32, epochs=50, val_every=250, lr=1e-4, size=227, save_freq=10)
model = ResNet
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=args.gamma)
test_ap, test_map = trainer.train(args, model, optimizer, scheduler, model_name='runs/q3/model')
print('test map:', test_map)

Train Epoch: 0 [0 (0%)]	Loss: 0.698527
Train Epoch: 0 [100 (64%)]	Loss: 0.254603
Train Epoch: 1 [200 (27%)]	Loss: 0.217977
Train Epoch: 1 [300 (91%)]	Loss: 0.197584
Train Epoch: 2 [400 (55%)]	Loss: 0.195816
Train Epoch: 3 [500 (18%)]	Loss: 0.203704
Train Epoch: 3 [600 (82%)]	Loss: 0.200449
Train Epoch: 4 [700 (46%)]	Loss: 0.182807
Train Epoch: 5 [800 (10%)]	Loss: 0.181640
Train Epoch: 5 [900 (73%)]	Loss: 0.181819
Train Epoch: 6 [1000 (37%)]	Loss: 0.226297
Train Epoch: 7 [1100 (1%)]	Loss: 0.184711
Train Epoch: 7 [1200 (64%)]	Loss: 0.172565
Train Epoch: 8 [1300 (28%)]	Loss: 0.176703
Train Epoch: 8 [1400 (92%)]	Loss: 0.185901
Train Epoch: 9 [1500 (55%)]	Loss: 0.135583
Train Epoch: 10 [1600 (19%)]	Loss: 0.179639
Train Epoch: 10 [1700 (83%)]	Loss: 0.153178
Train Epoch: 11 [1800 (46%)]	Loss: 0.157383
Train Epoch: 12 [1900 (10%)]	Loss: 0.158471
Train Epoch: 12 [2000 (74%)]	Loss: 0.168515
Train Epoch: 13 [2100 (38%)]	Loss: 0.169390
Train Epoch: 14 [2200 (1%)]	Loss: 0.130409
Train Epoch: 14 [23

![title](imgs/q3_tb_map.png)
![title](imgs/q3_tb_loss.png)

![title](imgs/q3_tb_lr.png)
![title](imgs/q3_tb_hist.png)